In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
torch.cuda.is_available()

True

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:02<00:00, 81235878.84it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Definicja pierwszej warstwy konwolucyjnej: wejście 3 kanały (RGB), wyjście 32 kanały, rozmiar filtra 3x3, padding=1
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # Definicja drugiej warstwy konwolucyjnej: wejście 32 kanały, wyjście 64 kanały, rozmiar filtra 3x3, padding=1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # Definicja trzeciej warstwy konwolucyjnej: wejście 64 kanały, wyjście 128 kanały, rozmiar filtra 3x3, padding=1
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # Warstwa dropout1 z prawdopodobieństwem wyzerowania 25% neuronów
        self.dropout1 = nn.Dropout(0.25)
        # Warstwa dropout2 z prawdopodobieństwem wyzerowania 50% neuronów
        self.dropout2 = nn.Dropout(0.5)
        # Warstwa w pełni połączona (fully connected), wejście 128 * 4 * 4 neurony (wynik rozmiaru tensora po ostatniej konwolucji), wyjście 512 neurony
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        # Warstwa w pełni połączona (fully connected), wejście 512 neurony, wyjście 10 neurony (liczba klas)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Przekształcenie wejścia przez pierwszą warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv1(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Przekształcenie przez drugą warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv2(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Przekształcenie przez trzecią warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv3(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Wykorzystanie warstwy dropout1
        x = self.dropout1(x)
        # Spłaszczenie tensora do postaci wektora przed podaniem na warstwę w pełni połączoną
        x = torch.flatten(x, 1)
        # Przekształcenie przez warstwę w pełni połączoną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.fc1(x))
        # Wykorzystanie warstwy dropout2
        x = self.dropout2(x)
        # Przekształcenie przez warstwę w pełni połączoną (wyjściową)
        x = self.fc2(x)
        # Zastosowanie funkcji log_softmax dla uzyskania prawdopodobieństw przynależności do klas
        output = nn.functional.log_softmax(x, dim=1)
        return output


In [5]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [6]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

[1, 100] loss: 2.280
[1, 200] loss: 2.069
[1, 300] loss: 1.943
[1, 400] loss: 1.856
[1, 500] loss: 1.750
[1, 600] loss: 1.690
[1, 700] loss: 1.642
[2, 100] loss: 1.564
[2, 200] loss: 1.528
[2, 300] loss: 1.493
[2, 400] loss: 1.476
[2, 500] loss: 1.459
[2, 600] loss: 1.435
[2, 700] loss: 1.379
[3, 100] loss: 1.335
[3, 200] loss: 1.302
[3, 300] loss: 1.270
[3, 400] loss: 1.294
[3, 500] loss: 1.246
[3, 600] loss: 1.211
[3, 700] loss: 1.221
[4, 100] loss: 1.161
[4, 200] loss: 1.147
[4, 300] loss: 1.132
[4, 400] loss: 1.162
[4, 500] loss: 1.077
[4, 600] loss: 1.086
[4, 700] loss: 1.103
[5, 100] loss: 1.041
[5, 200] loss: 1.022
[5, 300] loss: 0.999
[5, 400] loss: 1.020
[5, 500] loss: 1.031
[5, 600] loss: 1.027
[5, 700] loss: 0.991
[6, 100] loss: 0.976
[6, 200] loss: 0.950
[6, 300] loss: 0.964
[6, 400] loss: 0.954
[6, 500] loss: 0.943
[6, 600] loss: 0.949
[6, 700] loss: 0.918
[7, 100] loss: 0.947
[7, 200] loss: 0.904
[7, 300] loss: 0.897
[7, 400] loss: 0.921
[7, 500] loss: 0.912
[7, 600] loss

Accuracy on the test set: 74.14%
